# IEEE Clean Dataset

Copy clean wavs to /data directories.

# 1. Import libraries

In [20]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import math
import array

import re

import scipy.io as sio
import numpy as np
import sys

import time

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import matplotlib.pyplot as plt

print(sys.executable)
import librosa
import soundfile as sf
import librosa.display
import seaborn as sns

from skimage.restoration import unwrap_phase

/home/knayem/anaconda3/bin/python


# 2. Helping Functions (a) 

In [2]:
def replace_rev(string, old, new, times=1):
    '''
    Replace a substring (old) with another substring (new) from a string (string) 
    in total a fixed number (times) of times.
    '''
    
    ls = string.split(old)
    length = len(ls)
    
    # times can be atmost (length-1)
    times = times if (length-1)>=times else (length-1)
    
    new_string = old.join(ls[:length-times])
    
    for t in range(times,0,-1):
        new_string = new.join([new_string,ls[length-t]])
                               
    return new_string

# 3. Variables (a)

In [ ]:
TAG_QUANT = 'QUANT'
QUANTIZATION_LEVEL = 0.03125

# 4. Paths 

In [3]:
ROOT_PATH = '/data'
USER_PATH = 'knayem'

ROOT_USER_PATH = os.path.join(ROOT_PATH,USER_PATH)

### IEEE MALE Dataset

In [4]:
IEEE_MALE_CORPORA_PATH = os.path.join(ROOT_PATH,'SpeechCorpora/IEEE_male') # male

### IEEE FEMALE Dataset

In [43]:
IEEE_FEMALE_CORPORA_PATH = os.path.join(ROOT_PATH,'SpeechCorpora/IEEE_female') # female

In [44]:
# Train, Dev, Test Folders for Clean and Mixs
TRAIN_CLEAN_FOLDER = 'train_16k'
DEV_CLEAN_FOLDER = 'dev_16k'
TEST_CLEAN_PFOLDER = 'test_16k'

#### 4.a Clean

In [45]:
# CLEAN 16kHz PATH
# CLEAN_wavs_PATH = os.path.join(ROOT_USER_PATH,'IEEE_male_clean_16k') # male
CLEAN_wavs_PATH = os.path.join(ROOT_USER_PATH,'IEEE_female_clean_16k') # female

CLEAN_wavs_TRAIN_PATH = os.path.join(CLEAN_wavs_PATH, TRAIN_CLEAN_FOLDER)
CLEAN_wavs_DEV_PATH = os.path.join(CLEAN_wavs_PATH, DEV_CLEAN_FOLDER)
CLEAN_wavs_TEST_PATH = os.path.join(CLEAN_wavs_PATH, TEST_CLEAN_PFOLDER)

#### 4.b Mixture (Noisy)

In [ ]:
# .NPY FILE PATH
NPY_FILES_SAVE_PATH = 'IEEE_DataFiles' # store .npy data file path for quick access

# SSN PATH
SSN_wavs_PATH = replace_rev(CLEAN_wavs_PATH, "clean", "mix_SSN")
SSN_wavs_TRAIN_PATH = os.path.join(SSN_wavs_PATH, TRAIN_CLEAN_FOLDER)
SSN_wavs_DEV_PATH = os.path.join(SSN_wavs_PATH, DEV_CLEAN_FOLDER)
SSN_wavs_TEST_PATH = os.path.join(SSN_wavs_PATH, TEST_CLEAN_PFOLDER)

# CAFE PATH
CAFE_MIXTURE_PATH = replace_rev(CLEAN_wavs_PATH, "clean", "mix_Cafe")
CAFE_wavs_TRAIN_PATH = os.path.join(CAFE_MIXTURE_PATH, TRAIN_CLEAN_FOLDER)
CAFE_wavs_DEV_PATH = os.path.join(CAFE_MIXTURE_PATH, DEV_CLEAN_FOLDER)
CAFE_wavs_TEST_PATH = os.path.join(CAFE_MIXTURE_PATH, TEST_CLEAN_PFOLDER)

# BABBLE PATH
BABBLE_MIXTURE_PATH = replace_rev(CLEAN_wavs_PATH, "clean", "mix_Babble")
BABBLE_wavs_TRAIN_PATH = os.path.join(BABBLE_MIXTURE_PATH, TRAIN_CLEAN_FOLDER)
BABBLE_wavs_DEV_PATH = os.path.join(BABBLE_MIXTURE_PATH, DEV_CLEAN_FOLDER)
BABBLE_wavs_TEST_PATH = os.path.join(BABBLE_MIXTURE_PATH, TEST_CLEAN_PFOLDER)

# FACTORY PATH
FACTORY_MIXTURE_PATH = replace_rev(CLEAN_wavs_PATH, "clean", "mix_Factory")
FACTORY_wavs_TRAIN_PATH = os.path.join(FACTORY_MIXTURE_PATH, TRAIN_CLEAN_FOLDER)
FACTORY_wavs_DEV_PATH = os.path.join(FACTORY_MIXTURE_PATH, DEV_CLEAN_FOLDER)
FACTORY_wavs_TEST_PATH = os.path.join(FACTORY_MIXTURE_PATH, TEST_CLEAN_PFOLDER)

#### 4.c Enhanced (De-noised)

In [ ]:
# Enhanced PATH
Enhanced_wavs_PATH = replace_rev(CLEAN_wavs_PATH, "clean", "enhanced_"+TAG_QUANT)

In [ ]:
# Test folders
SSN_Enhanced_FOLDER = 'SSN'
CAFE_Enhanced_FOLDER = 'Cafe'
BABBLE_Enhanced_FOLDER = 'Babble'
FACTORY_Enhanced_FOLDER = 'Factory'

In [ ]:
# Enhanced Test Mix wavs PATH
SSN_Enhanced_wavs_PATH = os.path.join(Enhanced_wavs_PATH, SSN_Enhanced_FOLDER)
FACTORY_Enhanced_wavs_PATH = os.path.join(Enhanced_wavs_PATH, CAFE_Enhanced_FOLDER)
BABBLE_Enhanced_wavs_PATH = os.path.join(Enhanced_wavs_PATH, BABBLE_Enhanced_FOLDER)
FACTORY_Enhanced_wavs_PATH = os.path.join(Enhanced_wavs_PATH, FACTORY_Enhanced_FOLDER)

#### Path Summaray

In [47]:
print("Root path,", ROOT_PATH)
print("\t|-> Root User path,", ROOT_USER_PATH)
print()
print("IEEE Male Data Corpora path,", IEEE_MALE_CORPORA_PATH)
print("IEEE Female Data Corpora path,", IEEE_FEMALE_CORPORA_PATH)
print()
print("Clean .WAV path,", CLEAN_wavs_PATH)
print("\t|-> Train Clean .WAV path,", CLEAN_wavs_TRAIN_PATH)
print("\t|-> Dev Clean .WAV path,", CLEAN_wavs_DEV_PATH)
print("\t|-> Test Clean .WAV path,", CLEAN_wavs_TEST_PATH)
print()
print("Mix SSN .WAV path,", SSN_wavs_PATH)
print("\t|-> Train Mix SSN .WAV path,", SSN_wavs_TRAIN_PATH)
print("\t|-> Dev Mix SSN .WAV path,", SSN_wavs_DEV_PATH)
print("\t|-> Test Mix SSN .WAV path,", SSN_wavs_TEST_PATH)
print()
print("Mix CAFE .WAV path,", CAFE_MIXTURE_PATH)
print("\t|-> Train Mix CAFE .WAV path,", CAFE_wavs_TRAIN_PATH)
print("\t|-> Dev Mix CAFE .WAV path,", CAFE_wavs_DEV_PATH)
print("\t|-> Test Mix CAFE .WAV path,", CAFE_wavs_TEST_PATH)
print()
print("Mix BABBLE .WAV path,", BABBLE_MIXTURE_PATH)
print("\t|-> Train Mix BABBLE .WAV path,", BABBLE_wavs_TRAIN_PATH)
print("\t|-> Dev Mix BABBLE .WAV path,", BABBLE_wavs_DEV_PATH)
print("\t|-> Test Mix BABBLE .WAV path,", BABBLE_wavs_TEST_PATH)
print()
print("Mix FACTORY .WAV path,", FACTORY_MIXTURE_PATH)
print("\t|-> Train Mix FACTORY .WAV path,", FACTORY_wavs_TRAIN_PATH)
print("\t|-> Dev Mix FACTORY .WAV path,", FACTORY_wavs_DEV_PATH)
print("\t|-> Test Mix FACTORY .WAV path,", FACTORY_wavs_TEST_PATH)
print()
print("Enhanced .WAV path,", Enhanced_wavs_PATH)
print("\t|-> SSN Enhanced .WAV path,", SSN_Enhanced_wavs_PATH)
print("\t|-> FACTORY Enhanced .WAV path,", FACTORY_Enhanced_wavs_PATH)
print("\t|-> BABBLE Enhanced .WAV path,", BABBLE_Enhanced_wavs_PATH)
print("\t|-> FACTORY Enhanced .WAV path,", FACTORY_Enhanced_wavs_PATH)

Root path, /data
	|-> Root User path, /data/knayem

IEEE Male Data Corpora path, /data/SpeechCorpora/IEEE_male
IEEE Female Data Corpora path, /data/SpeechCorpora/IEEE_female

Clean .WAV path, /data/knayem/IEEE_female_clean_16k
	|-> Train Clean .WAV path, /data/knayem/IEEE_female_clean_16k/train_16k
	|-> Dev Clean .WAV path, /data/knayem/IEEE_female_clean_16k/dev_16k
	|-> Test Clean .WAV path, /data/knayem/IEEE_female_clean_16k/test_16k



NameError: name 'SSN_wavs_PATH' is not defined

### TIMIT Dataset 

#### 2.a Mixture (Noisy)

#### Clean

In [ ]:
# PATH = os.path.join(CLEAN_PATH,TRAIN_CLEAN_PATH) # clean train
# PATH = os.path.join(CLEAN_PATH,DEV_CLEAN_PATH) # clean dev
PATH = os.path.join(CLEAN_PATH,TEST_CLEAN_PATH) # clean test

# 5. STFT

### 5.a Parameters 

Followings are the basic parameter for calculating STFT.

In [48]:
fs = int(16e3)

n_fft = 640
win_length = int(40e-3*fs) # librosa needs scalar value
overlap = int(20e-3*fs)
hop_length = win_length - overlap # librosa needs scalar value

NUMS_PRINTS = 10

print('window: {0}, noverlap: {1}, nfft: {2}, fs: {3}, hop_length: {4}'.
      format(win_length,overlap,n_fft,fs,hop_length))

window: 640, noverlap: 320, nfft: 640, fs: 16000, hop_length: 320


### 5.b STFT function

Calculate Magnitude and Group Delay of the PATH (train, dev, test of IEEE/TIMIT) to get an overview of the data.

In [49]:
def mag_gd_phase(filename, fs, n_fft, hop_length, win_length):
    
    y, sr = librosa.load(filename, sr=fs)
    s_stft = librosa.stft(y,n_fft,hop_length,win_length)
    mag, phase = librosa.magphase(s_stft)
    angle = np.angle(phase)

    unwrap_angle = np.unwrap(angle, axis=0) # freq, MATLAB implementation
    unwrap_angle_s = np.roll(unwrap_angle, 1, axis=0) # roll across freq
    unwrap_GD = np.angle(np.exp(1j*(unwrap_angle - unwrap_angle_s))) # paper implementation

    return mag, unwrap_GD, phase, angle

## 6. Save Clean 

Save clean wavs

In [56]:
def saveCleanWav(wav_file_name, default_wavs_path, clean_wavs_path, expected_sample_rate):
    '''
    <wav_file_name> string : file name of a wav file
    <default_wavs_path> string : folder path where corpora wav files are saved
    <clean_wavs_path> string : folder path where clean wav files will be saved
    <expected_sample_rate> int : expected sample rate of the signals, clean and mix.
    '''
    
    wav_file_path = os.path.join(default_wavs_path, wav_file_name)
    
    y, sr = librosa.load(wav_file_path, sr=expected_sample_rate)
    filename, extension = wav_file_name.split('.')
    
    if sr >= 1e3:
        sr_string = "".join([str(math.floor(sr/1e3)), "k"])
        filename = "_".join([filename, sr_string])
    
    filename = ".".join([filename,extension])
    
    # clean directory creation               
    if not os.path.exists(clean_wavs_path):
        print(False,clean_wavs_path)
        os.makedirs(clean_wavs_path)
    else:
        print(True,clean_wavs_path)
    
    save_clean_wav_path = os.path.join(clean_wavs_path,filename)
    # save file
    sf.write(save_clean_wav_path, y, int(sr))
    
    return save_clean_wav_path, filename

In [51]:
# corpora_path_list = [IEEE_MALE_CORPORA_PATH]
# n_train_dev_test_wavs = [ [500,110,110] ]

corpora_path_list = [IEEE_FEMALE_CORPORA_PATH]
n_train_dev_test_wavs = [ [500,110,110] ]

In [57]:
for enum1, (corpora, n_train_dev_test) in enumerate( zip(sorted(corpora_path_list),n_train_dev_test_wavs) ):
    print(enum1,"CORPORA:", corpora)
    n_train_dev_test = np.cumsum(n_train_dev_test)
    print(n_train_dev_test)
    
    for root, dirs, files in os.walk(corpora): 
        wav_files = list( filter(lambda x: x.split('.')[-1] == 'wav', files) )
        print("ROOT:",root, ", len(DIR):", len(dirs), ", len(FILES):",len(wav_files))
        
        # male
        #wav_files = list( filter(lambda x: re.match( r'[A-Z]_[0-9]{2}_[0-9]{2}.wav', x, re.M|re.I),wav_files) )
        # female
        wav_files = list( filter(lambda x: re.match( r'[A-Z][0-9]{2}[a-z][0-9]{2}.wav', x, re.M|re.I),wav_files) )
        print("len(FILES):",len(wav_files))
        

        for enum2, filename in enumerate(sorted(wav_files)):
            
            if enum2 < n_train_dev_test[0]: # Train
                save_clean_wav_path, filename = saveCleanWav(filename, root, CLEAN_wavs_TRAIN_PATH, fs)
            elif enum2 < n_train_dev_test[1]: # Dev
                save_clean_wav_path, filename = saveCleanWav(filename, root, CLEAN_wavs_DEV_PATH, fs)
            elif enum2 < n_train_dev_test[2]: # Test
                save_clean_wav_path, filename = saveCleanWav(filename, root, CLEAN_wavs_TEST_PATH, fs)
                
            print (enum2, save_clean_wav_path, filename)
            

0 CORPORA: /data/SpeechCorpora/IEEE_female
[500 610 720]
ROOT: /data/SpeechCorpora/IEEE_female , len(DIR): 0 , len(FILES): 724
len(FILES): 720
False /data/knayem/IEEE_female_clean_16k/train_16k
0 /data/knayem/IEEE_female_clean_16k/train_16k/l01s01_16k.wav l01s01_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
1 /data/knayem/IEEE_female_clean_16k/train_16k/l01s02_16k.wav l01s02_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
2 /data/knayem/IEEE_female_clean_16k/train_16k/l01s03_16k.wav l01s03_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
3 /data/knayem/IEEE_female_clean_16k/train_16k/l01s04_16k.wav l01s04_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
4 /data/knayem/IEEE_female_clean_16k/train_16k/l01s05_16k.wav l01s05_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
5 /data/knayem/IEEE_female_clean_16k/train_16k/l01s06_16k.wav l01s06_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
6 /data/knayem/IEEE_female_clean_16k/train_1

True /data/knayem/IEEE_female_clean_16k/train_16k
63 /data/knayem/IEEE_female_clean_16k/train_16k/l07s04_16k.wav l07s04_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
64 /data/knayem/IEEE_female_clean_16k/train_16k/l07s05_16k.wav l07s05_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
65 /data/knayem/IEEE_female_clean_16k/train_16k/l07s06_16k.wav l07s06_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
66 /data/knayem/IEEE_female_clean_16k/train_16k/l07s07_16k.wav l07s07_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
67 /data/knayem/IEEE_female_clean_16k/train_16k/l07s08_16k.wav l07s08_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
68 /data/knayem/IEEE_female_clean_16k/train_16k/l07s09_16k.wav l07s09_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
69 /data/knayem/IEEE_female_clean_16k/train_16k/l07s10_16k.wav l07s10_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
70 /data/knayem/IEEE_female_clean_16k/train_16k/l08s01

True /data/knayem/IEEE_female_clean_16k/train_16k
128 /data/knayem/IEEE_female_clean_16k/train_16k/l13s09_16k.wav l13s09_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
129 /data/knayem/IEEE_female_clean_16k/train_16k/l13s10_16k.wav l13s10_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
130 /data/knayem/IEEE_female_clean_16k/train_16k/l14s01_16k.wav l14s01_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
131 /data/knayem/IEEE_female_clean_16k/train_16k/l14s02_16k.wav l14s02_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
132 /data/knayem/IEEE_female_clean_16k/train_16k/l14s03_16k.wav l14s03_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
133 /data/knayem/IEEE_female_clean_16k/train_16k/l14s04_16k.wav l14s04_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
134 /data/knayem/IEEE_female_clean_16k/train_16k/l14s05_16k.wav l14s05_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
135 /data/knayem/IEEE_female_clean_16k/train_16

192 /data/knayem/IEEE_female_clean_16k/train_16k/l20s03_16k.wav l20s03_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
193 /data/knayem/IEEE_female_clean_16k/train_16k/l20s04_16k.wav l20s04_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
194 /data/knayem/IEEE_female_clean_16k/train_16k/l20s05_16k.wav l20s05_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
195 /data/knayem/IEEE_female_clean_16k/train_16k/l20s06_16k.wav l20s06_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
196 /data/knayem/IEEE_female_clean_16k/train_16k/l20s07_16k.wav l20s07_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
197 /data/knayem/IEEE_female_clean_16k/train_16k/l20s08_16k.wav l20s08_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
198 /data/knayem/IEEE_female_clean_16k/train_16k/l20s09_16k.wav l20s09_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
199 /data/knayem/IEEE_female_clean_16k/train_16k/l20s10_16k.wav l20s10_16k.wav
True /data/knayem/

True /data/knayem/IEEE_female_clean_16k/train_16k
256 /data/knayem/IEEE_female_clean_16k/train_16k/l26s07_16k.wav l26s07_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
257 /data/knayem/IEEE_female_clean_16k/train_16k/l26s08_16k.wav l26s08_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
258 /data/knayem/IEEE_female_clean_16k/train_16k/l26s09_16k.wav l26s09_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
259 /data/knayem/IEEE_female_clean_16k/train_16k/l26s10_16k.wav l26s10_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
260 /data/knayem/IEEE_female_clean_16k/train_16k/l27s01_16k.wav l27s01_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
261 /data/knayem/IEEE_female_clean_16k/train_16k/l27s02_16k.wav l27s02_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
262 /data/knayem/IEEE_female_clean_16k/train_16k/l27s03_16k.wav l27s03_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
263 /data/knayem/IEEE_female_clean_16k/train_16

...

449 /data/knayem/IEEE_female_clean_16k/train_16k/l45s10_16k.wav l45s10_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
450 /data/knayem/IEEE_female_clean_16k/train_16k/l46s01_16k.wav l46s01_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
451 /data/knayem/IEEE_female_clean_16k/train_16k/l46s02_16k.wav l46s02_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
452 /data/knayem/IEEE_female_clean_16k/train_16k/l46s03_16k.wav l46s03_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
453 /data/knayem/IEEE_female_clean_16k/train_16k/l46s04_16k.wav l46s04_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
454 /data/knayem/IEEE_female_clean_16k/train_16k/l46s05_16k.wav l46s05_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
455 /data/knayem/IEEE_female_clean_16k/train_16k/l46s06_16k.wav l46s06_16k.wav
True /data/knayem/IEEE_female_clean_16k/train_16k
456 /data/knayem/IEEE_female_clean_16k/train_16k/l46s07_16k.wav l46s07_16k.wav
True /data/knayem/

True /data/knayem/IEEE_female_clean_16k/dev_16k
515 /data/knayem/IEEE_female_clean_16k/dev_16k/l52s06_16k.wav l52s06_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
516 /data/knayem/IEEE_female_clean_16k/dev_16k/l52s07_16k.wav l52s07_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
517 /data/knayem/IEEE_female_clean_16k/dev_16k/l52s08_16k.wav l52s08_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
518 /data/knayem/IEEE_female_clean_16k/dev_16k/l52s09_16k.wav l52s09_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
519 /data/knayem/IEEE_female_clean_16k/dev_16k/l52s10_16k.wav l52s10_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
520 /data/knayem/IEEE_female_clean_16k/dev_16k/l53s01_16k.wav l53s01_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
521 /data/knayem/IEEE_female_clean_16k/dev_16k/l53s02_16k.wav l53s02_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
522 /data/knayem/IEEE_female_clean_16k/dev_16k/l53s03_16k.wav l53s03_16k.wav


581 /data/knayem/IEEE_female_clean_16k/dev_16k/l59s02_16k.wav l59s02_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
582 /data/knayem/IEEE_female_clean_16k/dev_16k/l59s03_16k.wav l59s03_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
583 /data/knayem/IEEE_female_clean_16k/dev_16k/l59s04_16k.wav l59s04_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
584 /data/knayem/IEEE_female_clean_16k/dev_16k/l59s05_16k.wav l59s05_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
585 /data/knayem/IEEE_female_clean_16k/dev_16k/l59s06_16k.wav l59s06_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
586 /data/knayem/IEEE_female_clean_16k/dev_16k/l59s07_16k.wav l59s07_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
587 /data/knayem/IEEE_female_clean_16k/dev_16k/l59s08_16k.wav l59s08_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k
588 /data/knayem/IEEE_female_clean_16k/dev_16k/l59s09_16k.wav l59s09_16k.wav
True /data/knayem/IEEE_female_clean_16k/dev_16k


True /data/knayem/IEEE_female_clean_16k/test_16k
647 /data/knayem/IEEE_female_clean_16k/test_16k/l65s08_16k.wav l65s08_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
648 /data/knayem/IEEE_female_clean_16k/test_16k/l65s09_16k.wav l65s09_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
649 /data/knayem/IEEE_female_clean_16k/test_16k/l65s10_16k.wav l65s10_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
650 /data/knayem/IEEE_female_clean_16k/test_16k/l66s01_16k.wav l66s01_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
651 /data/knayem/IEEE_female_clean_16k/test_16k/l66s02_16k.wav l66s02_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
652 /data/knayem/IEEE_female_clean_16k/test_16k/l66s03_16k.wav l66s03_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
653 /data/knayem/IEEE_female_clean_16k/test_16k/l66s04_16k.wav l66s04_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
654 /data/knayem/IEEE_female_clean_16k/test_16k/l66s05_16k.wav

True /data/knayem/IEEE_female_clean_16k/test_16k
712 /data/knayem/IEEE_female_clean_16k/test_16k/l72s03_16k.wav l72s03_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
713 /data/knayem/IEEE_female_clean_16k/test_16k/l72s04_16k.wav l72s04_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
714 /data/knayem/IEEE_female_clean_16k/test_16k/l72s05_16k.wav l72s05_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
715 /data/knayem/IEEE_female_clean_16k/test_16k/l72s06_16k.wav l72s06_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
716 /data/knayem/IEEE_female_clean_16k/test_16k/l72s07_16k.wav l72s07_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
717 /data/knayem/IEEE_female_clean_16k/test_16k/l72s08_16k.wav l72s08_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
718 /data/knayem/IEEE_female_clean_16k/test_16k/l72s09_16k.wav l72s09_16k.wav
True /data/knayem/IEEE_female_clean_16k/test_16k
719 /data/knayem/IEEE_female_clean_16k/test_16k/l72s10_16k.wav